In [1]:
import numpy as np
from env.Environment import Environment, State

%load_ext autoreload
%autoreload 2

In [33]:
num_bed = 10
num_class = 6

In [34]:
p_arrive = [0.8988013698630137, 0.012994672754946727, 0.01293759512937595, 0.01221461187214612, 0.013032724505327244, 0.013089802130898021, 0.011910197869101979, 0.012271689497716894, 0.012747336377473363]
p_leave = [0.010825796872474158, 0.0038442559604472373, 0.00335442100653245, 0.0027426663004428303, 0.0026808956939254076, 0.0026285227575753496, 0.002433672892274736, 0.0018672590670831529]
delta = [0, 0.00749376480848956, 0.023252303608424292, 0.38966471920753987, 0.1, 0.04492930254495119, 0.019348297623417, 1.1000634771910827, 1.080419000471227]
# print(p_arrive, p_leave, delta)
env = Environment(num_class, num_bed, p_arrive, p_leave, delta)
env.calc_dynamics()

  0%|                                                                             | 15/393822 [00:00<45:56, 142.86it/s]

Total states: 393822
Calculating dynamics...


  1%|▍                                                                         | 2273/393822 [00:25<1:12:15, 90.32it/s]


KeyboardInterrupt: 

In [4]:
tmp = np.tile(np.arange(num_bed + 1), (num_class, 1))
tmp = np.array(np.meshgrid(*tmp)).T.reshape(-1, num_class)
all_x = tmp[tmp.sum(-1) <= num_bed]
num_rows = len(all_x)
# col_x = np.tile(all_x, (num_class + 1, 1))

# all_y = np.arange(num_class + 1)
# col_y = np.tile(all_y, (num_rows, 1)).T.reshape(-1, 1)
# all_states = np.concatenate((col_x, col_y), axis=-1)

In [44]:
num_rows

53130

In [47]:
tmp = np.tile(all_x, (num_bed+1,1))
print(tmp.shape)
tmp2 = np.tile(np.arange(num_bed + 1), (len(all_x), 1)).T.reshape(-1,1)
tmp = np.concatenate([tmp,tmp2], axis=-1)
all_x = tmp[tmp.sum(-1) <= num_bed]
all_x.shape

(18648630, 7)


(3108105, 8)